In [5]:
import os
from pyspark.sql import SparkSession

os.environ["PYSPARK_PYTHON"] = "/opt/miniconda3/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/opt/miniconda3/bin/python"

spark = SparkSession.builder \
    .appName("User_B_Session") \
    .master("spark://10.67.22.135:7077") \
    .config("spark.scheduler.mode", "FAIR") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.shuffle.service.enabled", "false") \
    .config("spark.scheduler.pool", "user_b") \
    .getOrCreate()


25/07/02 17:53:12 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [4]:
sc = spark.sparkContext

print("✅ SparkContext initialized.")

# Create an RDD with 8 partitions to make sure all workers get work
num_partitions = 8
data = list(range(1, 10001))
rdd = sc.parallelize(data, num_partitions)

print(f"✅ Created RDD with {rdd.getNumPartitions()} partitions")

# Force a distributed job with a shuffle
word_pairs = rdd.map(lambda x: (x % 4, 1))  # 4 keys, but partitions = 8
counts = word_pairs.reduceByKey(lambda a, b: a + b)

result = counts.collect()
print("✅ Result of reduceByKey:", result)

# Optional: also run DataFrame work
df = spark.createDataFrame([(i, f"Name{i}") for i in range(10)], ["id", "name"])
df.show()

✅ SparkContext initialized.
✅ Created RDD with 8 partitions
✅ Result of reduceByKey: [(0, 2500), (1, 2500), (2, 2500), (3, 2500)]
+---+-----+
| id| name|
+---+-----+
|  0|Name0|
|  1|Name1|
|  2|Name2|
|  3|Name3|
|  4|Name4|
|  5|Name5|
|  6|Name6|
|  7|Name7|
|  8|Name8|
|  9|Name9|
+---+-----+



Se la cella di sopra ha prodotto un risultato del tipo:\
✅ SparkContext initialized.\
✅ Created RDD with 8 partitions\
✅ Result of reduceByKey: [(0, 2500), (1, 2500), (2, 2500), (3, 2500)]\
+---+-----+
| id| name|
+---+-----+
|  0|Name0|
|  1|Name1|
|  2|Name2|
|  3|Name3|
|  4|Name4|
|  5|Name5|
|  6|Name6|
|  7|Name7|
|  8|Name8|
|  9|Name9|
+---+-----+\
Allora siamo a cavallo, dovreste vedere nella pagina del locahost User session il lavoro distribuito su 11 cores.

In [8]:
spark.stop()